## Code to produce "Model Schematic" Diagram used in 2023 Internal Lysis paper

In [1]:
import black
import jupyter_black

jupyter_black.load(
    lab=True,
    line_length=110,
    target_version=black.TargetVersion.PY310,
)

In [17]:
import datetime
import math
import os
import re

import numpy as np
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import pandas as pd

from matplotlib.animation import FFMpegWriter, FuncAnimation
from matplotlib.colors import BoundaryNorm, ListedColormap, Normalize

import lysis

pd.reset_option("display.precision")
pd.set_option("display.float_format", lambda x: f"{x:,.3f}")

In [3]:
scenario_type = np.dtype(
    [
        ("descriptor", np.str_, 40),
        ("total_molecules", int),
        ("pore_size", float),  # in microns. Code requires cm
        ("fiber_diameter", float),  # in microns. Needs to be added to pore_size to get grid_node_distance
        ("cols", int),
        ("rows", int),
        ("empty_rows", int),
        ("forced_unbind", float),
    ]
)
mechanism_type = np.dtype([("descriptor", np.str_, 40), ("executable", np.str_, 40)])
run_type = np.dtype(
    [
        ("exp_code", np.str_, 15),
        ("scenario", np.str_, 40),
        ("mechanism", np.str_, 40),
        ("seed", int),
        ("running_time", int),
    ]
)
diameter_code = {
    72.7: "Q2",
    145.4: "Q4",
}

In [4]:
# seed sequence entropy: 3881821051554698152964433817123076384
scenarios = np.array(
    [
        ("Demo_0", 1000, 0.22, 145.4, 40, 60, 20, 0.0852),
        ("Demo_1", 500, 0.22, 145.4, 40, 60, 20, 0.0852),
        ("Demo_2", 1000, 0.22, 145.4, 20, 30, 10, 0.0852),
        ("Demo_3", 500, 0.22, 145.4, 20, 30, 10, 0.0852),
        ("Demo_4", 500, 0.22, 145.4, 15, 20, 5, 0.0852),
        ("Demo_5", 200, 0.22, 145.4, 15, 20, 5, 0.0852),
    ],
    dtype=scenario_type,
)
mechanisms = np.array(
    [
        ("Into and along - Internal", "macro_diffuse_into_and_along__internal"),
    ],
    dtype=mechanism_type,
)

In [5]:
in_file_code = "_PLG2_tPA01_{input_code}.dat"
out_file_code = "_{scenario_code}"

# slope_tolerance = 1e-3
rng = np.random.default_rng(65463453)

In [6]:
group_code = "2023-05-17-14"
runs = np.array(
    [
        # (group_code + "00", "Demo_0", "Into and along - Internal", 1_034_836_197, 0),
        # (group_code + "01", "Demo_1", "Into and along - Internal", 2_464_079_704, 0),
        # (group_code + "02", "Demo_2", "Into and along - Internal", 982_919_027, 0),
        # (group_code + "03", "Demo_3", "Into and along - Internal", 129_314_213, 0),
        (group_code + "04", "Demo_4", "Into and along - Internal", 2_705_873_059, 0),
        # (group_code + "05", "Demo_5", "Into and along - Internal", 2_104_158_012, 0),
    ],
    dtype=run_type,
)

In [7]:
index = pd.Index(runs["scenario"], name="Scenario")
degrade_percent_markers = [0.05, 0.2, 0.5, 0.8]
columns = pd.MultiIndex.from_product(
    [[f"{int(i*100)}%" for i in degrade_percent_markers], range(10)], names=["Percent Degraded", "Simulation"]
)
degradation_time_df = pd.DataFrame(index=index, columns=columns)

degrade_slope_between_percent = [[0.2, 0.8], [0.2, 0.5], [0.5, 0.8]]
columns = pd.MultiIndex.from_product(
    [[f"{int(i[0]*100)}% to {int(i[1]*100)}%" for i in degrade_slope_between_percent], range(10)],
    names=["Degradation Interval", "Simulation"],
)
slopes_df = pd.DataFrame(index=index, columns=columns)

In [8]:
# For a light colormap of degraded fibers
colormap_f = plt.cm.cividis(np.arange(plt.cm.cividis.N))
colormap_f[:, 3] = 0.3
# For very light grey degraded fibers
colormap_f = ["xkcd:light grey"]

colormap_f = ListedColormap(colormap_f)
colormap_f.set_extremes(bad="xkcd:blue", under="white", over="xkcd:blue")
colormap_m = ListedColormap(["xkcd:green", "xkcd:cyan"])
bounds = [0, 0.5, 1]
norm_m = BoundaryNorm(bounds, colormap_m.N)

angle = (1 / 6) * math.pi
radius = 0.5
edge_width = 8

font = {
    "color": "black",
    "weight": "bold",
    "style": "normal",
    "size": 20,
}

In [9]:
def load_fortran_files(exp, file_code):
    n_save = np.asarray(
        [
            np.fromfile(
                os.path.join(e.os_path, f"{sim:02}", f"Nsave{file_code[:-4]}_{sim:02}{file_code[-4:]}"),
                dtype=np.int32,
            )[0]
            for sim in range(e.macro_params.total_trials)
        ]
    )
    n_save += 1
    n_save

    tsave = [
        np.fromfile(os.path.join(e.os_path, f"{sim:02}", f"tsave{file_code[:-4]}_{sim:02}{file_code[-4:]}"))
        for sim in range(e.macro_params.total_trials)
    ]
    tmax = max([np.max(arr) for arr in tsave])

    mfpt = np.asarray(
        [
            np.fromfile(
                os.path.join(e.os_path, f"{sim:02}", f"mfpt{file_code[:-4]}_{sim:02}{file_code[-4:]}")
            )
            for sim in range(e.macro_params.total_trials)
        ]
    )

    mol_location = []
    mol_status = []
    mapped_deg = []
    for sim in range(e.macro_params.total_trials):
        raw_mol_location = np.fromfile(
            os.path.join(e.os_path, f"{sim:02}", f"m_loc{file_code[:-4]}_{sim:02}{file_code[-4:]}"),
            dtype=np.int32,
        )
        mol_location.append(raw_mol_location.reshape(n_save[sim], e.macro_params.total_molecules) - 1)

        raw_mol_status = np.fromfile(
            os.path.join(e.os_path, f"{sim:02}", f"m_bound{file_code[:-4]}_{sim:02}{file_code[-4:]}"),
            dtype=np.int32,
        )
        raw_mol_status = raw_mol_status.astype(np.bool_)
        mol_status.append(raw_mol_status.reshape(n_save[sim], e.macro_params.total_molecules))

        raw_deg = np.fromfile(
            os.path.join(e.os_path, f"{sim:02}", f"f_deg_time{file_code[:-4]}_{sim:02}{file_code[-4:]}")
        )
        # raw_mapped_deg[raw_deg == 0] = tmax + e.macro_params.save_interval  # float('inf') #
        mapped_deg.append(raw_deg.reshape(n_save[sim], e.macro_params.total_edges))

    return n_save, mapped_deg, tsave, mfpt, mol_location, mol_status


def find_degraded_fraction(exp, deg, tsave):
    degraded_fraction = []
    for r in range(exp.macro_params.total_trials):
        run_degraded_fraction = np.empty(deg[r].shape[0], dtype=np.float_)
        for t in range(deg[r].shape[0]):
            run_degraded_fraction[t] = np.count_nonzero(deg[r][t] <= tsave[r][t])
        run_degraded_fraction -= exp.macro_params.empty_rows * exp.macro_params.full_row
        degraded_fraction.append(run_degraded_fraction / exp.macro_params.total_fibers)
    return degraded_fraction


def find_degradation_marker_frames(exp, degraded_fraction):
    degrade_marker_frames = np.empty(
        (exp.macro_params.total_trials, len(degrade_percent_markers)), dtype=np.int_
    )
    for sim in range(exp.macro_params.total_trials):
        for marker in range(len(degrade_percent_markers)):
            degrade_marker_frames[sim, marker] = np.argmax(
                degraded_fraction[sim] >= degrade_percent_markers[marker]
            )
    return degrade_marker_frames


# Changed to seconds
def find_degradation_marker_times(exp, degrade_marker_frames, tsave):
    degradation_marker_times = np.empty(
        (exp.macro_params.total_trials, len(degrade_percent_markers)), dtype=np.float_
    )
    for sim in range(exp.macro_params.total_trials):
        degradation_marker_times[sim] = tsave[sim][degrade_marker_frames[sim]]
    return degradation_marker_times  # / 60

In [10]:
def plot_coords(i, j):
    x = j
    y = -i
    if j % 3 == 0:
        return x / 3.0, y - 0.5
    if j % 3 == 1:
        return (x - 1) / 3.0, y
    if j % 3 == 2:
        return (x - 2) / 3.0 + 0.5, y


def get_edge_index(exp):
    edge_index = np.empty(exp.macro_params.total_edges, dtype=tuple)
    for k in range(exp.macro_params.total_edges):
        edge_index[k] = lysis.from_fortran_edge_index(k, exp.macro_params.rows, exp.macro_params.cols)
    return edge_index


def animation_data(exp, edge_index, mol_location):
    x_f = np.empty(exp.macro_params.total_edges, dtype=float)
    y_f = np.empty(exp.macro_params.total_edges, dtype=float)
    for k in range(exp.macro_params.total_edges):
        i, j = edge_index[k]
        x_f[k], y_f[k] = plot_coords(i, j)
    d_x = (rng.random(size=exp.macro_params.total_molecules) - 0.5) / 2.5
    d_y = (rng.random(size=exp.macro_params.total_molecules) - 0.5) / 2.5
    x_m = []
    y_m = []
    for r in range(exp.macro_params.total_trials):
        x = np.empty((n_save[r], exp.macro_params.total_molecules), dtype=np.float_)
        y = np.empty((n_save[r], exp.macro_params.total_molecules), dtype=np.float_)
        for t in range(n_save[r]):
            for k in range(exp.macro_params.total_molecules):
                i, j = edge_index[mol_location[r][t, k]]
                x[t, k], y[t, k] = plot_coords(i, j)
            x[t] += d_x
            y[t] += d_y
        x_m.append(x)
        y_m.append(y)
    return x_f, y_f, x_m, y_m


def create_animation(exp, file_code, edge_index, deg, mol_status, tsave, x_f, y_f, x_m, y_m):
    for sim in range(1):
        fig = plt.figure(figsize=(exp.macro_params.cols / 10, exp.macro_params.rows / 10), dpi=100)
        ax = fig.add_axes([0, 0, 1, 1])
        ax.set_axis_off()
        vmin = 0
        vmax = exp.macro_params.cols - 1

        ydiff = exp.macro_params.rows - exp.macro_params.cols

        ax.set_xlim(vmin - 0.25, vmax + 0.25)
        ax.set_ylim(-vmax - ydiff - 0.25, -vmin + 0.25)
        ax.set_aspect("equal")
        fig.canvas.draw()
        title = ax.annotate(
            f"Time elapsed: {str(datetime.timedelta(seconds=tsave[sim][0])).split('.')[0]}",
            (0, 0),
            zorder=100,
            va="top",
        )
        s = (ax.get_window_extent().width / (vmax - vmin + 0.5) * 72 / (2 * fig.dpi)) ** 2
        scatt_f = ax.scatter(
            x_f,
            y_f,
            s=s,
            marker="s",
            linewidths=0,
            c=deg[sim][0],
            cmap=colormap_f,
            vmin=exp.macro_params.time_step,
            vmax=exp.macro_params.time_step,
        )
        scatt_m = ax.scatter(
            x_m[sim][0],
            y_m[sim][0],
            s=s / 2.5,
            marker="o",
            linewidths=0,
            c=mol_status[sim][0],
            cmap=colormap_m,
            norm=norm_m,
        )

        def update(frame_number):
            scatt_f.set_array(deg[sim][frame_number])
            if tsave[sim][frame_number] == 0:
                scatt_f.set_clim(vmin=exp.macro_params.time_step, vmax=exp.macro_params.time_step)
            else:
                scatt_f.set_clim(vmin=exp.macro_params.time_step, vmax=tsave[sim][frame_number])
            scatt_m.set_array(mol_status[sim][frame_number])
            scatt_m.set_offsets(np.append((x_m[sim][frame_number],), (y_m[sim][frame_number],), axis=0).T)
            title.set_text(
                f"Time elapsed: {str(datetime.timedelta(seconds=tsave[sim][frame_number])).split('.')[0]}"
            )

        animation = FuncAnimation(fig, update, frames=np.arange(n_save[sim]), interval=200)

        FFwriter = FFMpegWriter(fps=10)
        animation.save(
            os.path.join(e.os_path, f"{sim:02}", f"combined_animation_{sim:02}" + file_code[:-4] + ".mp4"),
            writer=FFwriter,
        )
        plt.close()


def create_animation_stills(
    exp, file_code, degrade_marker_frames, edge_index, deg, mol_status, tsave, x_f, y_f, x_m, y_m
):
    for sim in range(1):
        for i, frame in enumerate(degrade_marker_frames[sim]):
            fig = plt.figure(figsize=(exp.macro_params.cols / 10, exp.macro_params.rows / 10), dpi=100)
            ax = fig.add_axes([0, 0, 1, 1])
            ax.set_axis_off()
            vmin = 0
            vmax = exp.macro_params.cols - 1

            ydiff = exp.macro_params.rows - exp.macro_params.cols

            ax.set_xlim(vmin - 0.25, vmax + 0.25)
            ax.set_ylim(-vmax - ydiff - 0.25, -vmin + 0.25)
            ax.set_aspect("equal")
            fig.canvas.draw()
            title = ax.annotate(
                f"Simulation time elapsed: {str(datetime.timedelta(seconds=tsave[sim][frame])).split('.')[0]}",
                (0, 0),
                zorder=100,
                va="top",
            )
            s = (ax.get_window_extent().width / (vmax - vmin + 0.5) * 72 / (2 * fig.dpi)) ** 2
            scatt_f = ax.scatter(
                x_f,
                y_f,
                s=s,
                marker="s",
                linewidths=0,
                c=deg[sim][frame],
                cmap=colormap_f,
                vmin=exp.macro_params.time_step,
                vmax=tsave[sim][frame],
            )
            scatt_m = ax.scatter(
                x_m[sim][frame],
                y_m[sim][frame],
                s=s / 2.5,
                marker="o",
                linewidths=0,
                c=mol_status[sim][frame],
                cmap=colormap_m,
                norm=norm_m,
            )

            fig.savefig(
                os.path.join(
                    e.os_path,
                    f"{sim:02}",
                    f"experiment_state_plot_r{sim:02}_{degrade_percent_markers[i]:.2f}"
                    + file_code[:-4]
                    + ".png",
                ),
                bbox_inches="tight",
            )
            plt.close()

In [32]:
def fiber_end_coords(i, j):
    x1 = j // 3
    y1 = i
    if j % 3 == 0:
        x2 = x1
        y2 = y1 + 2 * radius
    elif j % 3 == 1:
        x2 = x1 - radius * math.cos(angle)
        y2 = y1 + radius * math.sin(angle)
        # x1 = x1 - radius * math.cos(angle)
        # y1 = y1 + radius * math.sin(angle)
    elif j % 3 == 2:
        x2 = x1 + 2 * radius
        y2 = y1
    return x1, y1, x2, y2


def grid_data(exp, edge_index):
    x1_f = np.empty(exp.macro_params.total_edges, dtype=float)
    y1_f = np.empty(exp.macro_params.total_edges, dtype=float)
    x2_f = np.empty(exp.macro_params.total_edges, dtype=float)
    y2_f = np.empty(exp.macro_params.total_edges, dtype=float)
    for k in range(exp.macro_params.total_edges):
        i, j = edge_index[k]
        x1_f[k], y1_f[k], x2_f[k], y2_f[k] = fiber_end_coords(i, j)
    return x1_f, y1_f, x2_f, y2_f


def molecule_data(exp, sim, frame, edge_index, mol_location):
    b = 0.3
    d_l = rng.random(size=exp.macro_params.total_molecules) * (1 - b) + b / 2
    d_w = rng.random(size=exp.macro_params.total_molecules) * edge_width / 72
    x_m = np.empty(exp.macro_params.total_molecules, dtype=np.float_)
    y_m = np.empty(exp.macro_params.total_molecules, dtype=np.float_)
    for k in range(exp.macro_params.total_molecules):
        i, j = edge_index[mol_location[sim][frame, k]]
        x1, y1, x2, y2 = fiber_end_coords(i, j)
        x_m[k] = d_l[k] * x1 + (1 - d_l[k]) * x2
        y_m[k] = d_l[k] * y1 + (1 - d_l[k]) * y2
    return x_m, y_m


def create_model_schematic_plot(
    exp, file_code, edge_index, degrade_marker_frames, deg, tsave, mol_location, mol_status
):
    image_scale = (
        exp.macro_params.grid_node_distance
        / 72
        / (exp.macro_params.grid_node_distance - 10_000 * exp.macro_params.pore_size)
    )
    marker = ["o", None, "o"]
    markevery = [1, 2, 1]
    x1_f, y1_f, x2_f, y2_f = grid_data(exp, edge_index)
    for sim in range(1):
        for idx, frame in enumerate(degrade_marker_frames[sim]):
            fig = plt.figure(
                figsize=(
                    1 + exp.macro_params.cols * image_scale * 25,
                    exp.macro_params.rows * image_scale * 25,
                ),
                dpi=72,
            )
            scale = Normalize(
                vmin=exp.macro_params.time_step,
                vmax=tsave[sim][frame],
            )
            ax = fig.add_axes([0, 0, 1, 1])
            ax.set_axis_off()
            vmin = 0
            vmax = e.macro_params.cols - 1

            ydiff = e.macro_params.rows - e.macro_params.cols

            ax.set_xlim(vmin - 2, vmax + 1)
            ax.set_ylim(vmin - 1, vmax + ydiff + 1)
            ax.set_aspect("equal")
            fig.canvas.draw()
            handles = []

            # Draw fibers
            for k in range(e.macro_params.total_edges):
                i, j = edge_index[k]
                # ax.annotate(
                #     f"({i}, {j})",
                #     (0.5 * (x1_f[k] + x2_f[k]), 0.5 * (y1_f[k] + y2_f[k])),
                #     color="red",
                #     zorder=9999999999,
                # )
                if deg[sim][frame][k] > 0:
                    plt.plot(
                        [x1_f[k], x2_f[k]],
                        [y1_f[k], y2_f[k]],
                        color=colormap_f(scale(deg[sim][frame][k])),
                        marker="o",
                        markevery=markevery[j % 3],
                        markersize=1.5 * edge_width,
                        linewidth=edge_width,
                        zorder=math.log(deg[sim][frame][k]),
                    )
                    if j % 3 == 1:
                        plt.plot(
                            [x1_f[k], 2 * x1_f[k] - x2_f[k]],
                            [y1_f[k], 2 * y1_f[k] - y2_f[k]],
                            color=colormap_f(scale(deg[sim][frame][k])),
                            linewidth=edge_width,
                            linestyle=(0, (0.3, 0.3)),
                            zorder=1,
                        )
            handles.append(
                mlines.Line2D([], [], color=colormap_f(2), linewidth=edge_width, label="Intact Fiber")
            )
            handles.append(
                mlines.Line2D([], [], color=colormap_f(0.5), linewidth=edge_width, label="Degraded Fiber")
            )

            # Draw bounding box
            buffer = 0.2
            x = np.asarray(
                [
                    -buffer,
                    e.macro_params.cols - 1 + buffer,
                    e.macro_params.cols - 1 + buffer,
                    -buffer,
                    -buffer,
                ],
                dtype=float,
            )
            y = np.asarray(
                [
                    -buffer,
                    -buffer,
                    e.macro_params.rows - 1 + buffer,
                    e.macro_params.rows - 1 + buffer,
                    -buffer,
                ],
                dtype=float,
            )
            x_front = x - (1 + buffer) * radius * math.cos(angle)
            y_front = y + (1 + buffer) * radius * math.sin(angle)
            x_rear = x + (1 + buffer) * radius * math.cos(angle)
            y_rear = y - (1 + buffer) * radius * math.sin(angle)

            plt.plot(
                x_front,
                y_front,
                color="black",
                linewidth=0.5 * edge_width,
                zorder=9999999999,
            )
            plt.plot(
                x_rear,
                y_rear,
                color="black",
                linewidth=0.5 * edge_width,
                zorder=0,
            )
            for k in range(4):
                plt.plot(
                    [x_front[k], x_rear[k]],
                    [y_front[k], y_rear[k]],
                    color="black",
                    linewidth=0.5 * edge_width,
                    zorder=0,
                )

            # Draw molecules
            x_m, y_m = molecule_data(exp, sim, frame, edge_index, mol_location)
            scatt_m = ax.scatter(
                x_m,
                y_m,
                s=edge_width * 5,
                marker="o",
                label="Bound tPA Molecule",
                linewidths=0,
                c=mol_status[sim][frame],
                cmap=colormap_m,
                norm=norm_m,
                zorder=100_000,
            )
            handles.append(
                mlines.Line2D(
                    [],
                    [],
                    color=colormap_m(False),
                    linewidth=0,
                    markersize=edge_width,
                    marker="o",
                    label="Unbound tPA Molecule",
                )
            )
            handles.append(
                mlines.Line2D(
                    [],
                    [],
                    color=colormap_m(True),
                    linewidth=0,
                    markersize=edge_width,
                    marker="o",
                    label="Bound tPA Molecule",
                )
            )

            # Draw braces
            lysis.util.curlyBrace(
                fig,
                ax,
                [-1, e.macro_params.empty_rows],
                [-1, e.macro_params.rows - 1],
                k_r=0.02,
                str_text="Clot",
                color="black",
                linewidth=0.5 * edge_width,
                fontdict=font,
            )
            lysis.util.curlyBrace(
                fig,
                ax,
                [-1, 0],
                [-1, e.macro_params.empty_rows - 0.1],
                k_r=0.05,
                str_text="Fibrin-free region",
                color="black",
                linewidth=0.5 * edge_width,
                fontdict=font,
            )
            ax.legend(handles=handles, loc="upper right", bbox_to_anchor=(0.92, 0.95), fontsize=16)

            ax.invert_yaxis()

            fig.savefig(
                os.path.join(
                    e.os_path,
                    f"{sim:02}",
                    f"fiber_grid_plot_r{sim:02}_{degrade_percent_markers[idx]:.2f}" + file_code[:-4] + ".png",
                ),
                bbox_inches="tight",
            )
            plt.close()

In [33]:
for run in runs:
    mech = mechanisms[mechanisms["descriptor"] == run["mechanism"]][0]
    scen = scenarios[scenarios["descriptor"] == run["scenario"]][0]
    e = lysis.util.Experiment(os.path.join("..", "..", "data"), experiment_code=run["exp_code"])
    # e = lysis.util.Experiment(
    #     os.path.join("/", "home", "bpaynter", "Archive", "lysis_data"), experiment_code=run["exp_code"]
    # )
    e.read_file()
    y_distance = np.arange(e.macro_params.rows - 1) * e.macro_params.grid_node_distance
    file_code = out_file_code.format(scenario_code=run["scenario"]) + ".dat"
    print(run["exp_code"], file_code)
    n_save, deg, tsave, mfpt, mol_location, mol_status = load_fortran_files(e, file_code)

    deg_fraction = find_degraded_fraction(e, deg, tsave)
    marker_frames = find_degradation_marker_frames(e, deg_fraction)
    marker_times = find_degradation_marker_times(e, marker_frames, tsave)
    # degradation_time_df.loc[run["scenario"]] = marker_times.T.flatten()
    # slopes = degradation_rates(e, marker_frames, deg_fraction, tsave)
    # slopes_df.loc[run["scenario"]] = slopes.T.flatten()

    # save_degrade_fraction_table(e, file_code, deg_fraction, tsave)

    # plot_degradation_percent(e, deg_fraction, tsave, marker_frames, slopes)

    edge_index = get_edge_index(e)
    # x_f, y_f, x_m, y_m = animation_data(e, edge_index, mol_location)
    # create_animation(e, file_code, edge_index, deg, mol_status, tsave, x_f, y_f, x_m, y_m)
    # create_animation_stills(
    #     e, file_code, marker_frames, edge_index, deg, mol_status, tsave, x_f, y_f, x_m, y_m
    # )

    create_model_schematic_plot(e, file_code, edge_index, marker_frames, deg, tsave, mol_location, mol_status)

2023-05-17-1404 _Demo_4.dat


In [13]:
sim = 0
frame = marker_frames[sim][1]
scen = scenarios[scenarios["descriptor"] == run["scenario"]][0]
scen

('Demo_5', 200, 0.22, 145.4, 15, 20, 5, 0.0852)

In [14]:
marker_frames

array([[ 2,  4,  6, 10],
       [ 2,  3,  6,  9],
       [ 2,  3,  5,  9],
       [ 2,  3,  6,  9],
       [ 2,  3,  6,  9],
       [ 2,  3,  6,  9],
       [ 2,  3,  6,  9],
       [ 2,  3,  6,  9],
       [ 2,  3,  6,  9],
       [ 2,  3,  6,  9]])

In [29]:
tsave[sim][frame]

4200.000306006001

In [48]:
x = np.asarray([0, e.macro_params.cols - 1, e.macro_params.cols - 1, 0, 0], dtype=float)
x

array([ 0., 14., 14.,  0.,  0.])

In [3]:
lysis.util.curlyBrace()

TypeError: curlyBrace() missing 4 required positional arguments: 'fig', 'ax', 'p1', and 'p2'

In [95]:
mol_status[sim][frame, 13]

True

In [98]:
mol_location[sim][frame, 13]

382

In [99]:
deg[sim][frame, 382]

512.5678490801826

In [101]:
tsave[sim][frame]

400.0000092